# Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib


# Load And Process DATA

In [2]:
# Load the CSV
df = pd.read_csv("groundwater_yearly_averages.csv")

# Drop rows with missing important values
df.dropna(subset=['Station_name', 'lat', 'long', 'Year', 'level'], inplace=True)

# Encode station name
station_encoder = LabelEncoder()
df['Station_ID'] = station_encoder.fit_transform(df['Station_name'])

# Features: station ID, lat, long, year
X = df[['Station_ID', 'lat', 'long', 'Year']].values
y = df['level'].values

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# Split the data

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


# Build the Neural Network Model

In [4]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer: one continuous value
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=64)




Epoch 1/50


338/338 [==============================] - 4s 5ms/step - loss: 49.2338 - mae: 3.6081 - val_loss: 100.2306 - val_mae: 3.6613
Epoch 2/50
338/338 [==============================] - 1s 3ms/step - loss: 44.6206 - mae: 3.2878 - val_loss: 99.9346 - val_mae: 3.3604
Epoch 3/50
338/338 [==============================] - 1s 4ms/step - loss: 43.9452 - mae: 3.2192 - val_loss: 98.4774 - val_mae: 3.4128
Epoch 4/50
338/338 [==============================] - 1s 4ms/step - loss: 43.5066 - mae: 3.1743 - val_loss: 98.5424 - val_mae: 3.2910
Epoch 5/50
338/338 [==============================] - 1s 4ms/step - loss: 43.3848 - mae: 3.1557 - val_loss: 97.9629 - val_mae: 3.4601
Epoch 6/50
338/338 [==============================] - 1s 4ms/step - loss: 43.1830 - mae: 3.1448 - val_loss: 97.9689 - val_mae: 3.3343
Epoch 7/50
338/338 [==============================] - 1s 3ms/step - loss: 42.9935 - mae: 3.1278 - val_loss: 98.2006 - val_mae: 3.2841
Epoch 8/50
338/338 [==============================] - 1s 

# Save Model


In [5]:
model.save("groundwater_level_model.h5")
joblib.dump(station_encoder, "station_encoder.pkl")
joblib.dump(scaler, "scaler.pkl")


c:\Users\Devansh\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['scaler.pkl']

In [7]:
def predict_ground_level(station_name, year):
    # Load encoders and model
    import tensorflow as tf
    import joblib
    import numpy as np

    station_encoder = joblib.load("station_encoder.pkl")
    scaler = joblib.load("scaler.pkl")
    model = tf.keras.models.load_model("groundwater_level_model.h5")

    # Get lat/long of station
    station_row = df[df['Station_name'] == station_name].iloc[0]
    station_id = station_encoder.transform([station_name])[0]

    lat = station_row['lat']
    long = station_row['long']

    # Prepare input
    input_data = np.array([[station_id, lat, long, year]])
    input_scaled = scaler.transform(input_data)

    # Predict
    predicted_level = model.predict(input_scaled)[0][0]

    # Print all info
    print(f"📍 Station: {station_name}")
    print(f"🧭 Location: Latitude = {lat}, Longitude = {long}")
    print(f"📅 Year: {year}")
    print(f"💧 Predicted Groundwater Level: {predicted_level:.2f} m")

    return predicted_level, lat, long


In [8]:
predict_ground_level("Kothri (np)", 2029)


1/1 [==============================] - 0s 212ms/step
📍 Station: Kothri (np)
🧭 Location: Latitude = 23.06745944, Longitude = 76.84140611
📅 Year: 2029
💧 Predicted Groundwater Level: 25.43 m


(25.427393, 23.06745944, 76.84140611)